In [1]:
from jetstream_hugo.jet_finding import *
from jetstream_hugo.definitions import *
from jetstream_hugo.stats import *
from jetstream_hugo.plots import *
from jetstream_hugo.clustering import *
from jetstream_hugo.data import *
from jetstream_hugo.hotspells import *
from simpsom.plots import *
import colormaps

%load_ext autoreload
%autoreload 2
%matplotlib inline

ERROR 1: PROJ: proj_create_from_database: Open of /storage/homefs/hb22g102/mambaforge/envs/env11/share/proj failed


In [2]:
exp = MultiVarExperiment(
    "ERA5", "plev", ["u", "v", "s"], "6H", (1940, 2022), None, -80, 30, 20, 80, [200, 250, 300]
)
ds = exp.ds
all_jets, where_are_jets, all_jets_one_array, all_jets_over_time, flags = exp.track_jets()
props_as_ds_uncat = exp.props_as_ds(False)
props_as_ds = exp.props_as_ds(True)
props_as_ds["nao"] = get_nao(interp_like=props_as_ds)
time_mask = (props_as_ds.time.dt.season=="JJA") & (props_as_ds.time.dt.year>=1959)
da_jet_pos = jet_position_as_da(ds["s"], props_as_ds, props_as_ds_uncat, all_jets, exp.path).sel(time=time_mask)
ds_late = flatten_by(ds.sel(time=time_mask), "s")
subset = ["mean_lon", "mean_lat", "mean_lev", "width", "persistence", "int"]
props_as_np = props_as_ds[subset].sel(time=time_mask).to_dataarray().values
props_as_np = props_as_np.transpose(1, 0, 2).reshape(props_as_np.shape[1], props_as_np.shape[0] * props_as_np.shape[2]) 

In [10]:
all_jets_over_time

array([[[     0,      0],
        [     1,      1],
        [     2,      3],
        ...,
        [121264, 121264],
        [121264, 121264],
        [121264, 121264]],

       [[     0,      1],
        [     1,      0],
        [     2,      0],
        ...,
        [121264, 121264],
        [121264, 121264],
        [121264, 121264]],

       [[     1,      2],
        [     2,      2],
        [     3,      0],
        ...,
        [121264, 121264],
        [121264, 121264],
        [121264, 121264]],

       ...,

       [[121264, 121264],
        [121264, 121264],
        [121264, 121264],
        ...,
        [121264, 121264],
        [121264, 121264],
        [121264, 121264]],

       [[121264, 121264],
        [121264, 121264],
        [121264, 121264],
        ...,
        [121264, 121264],
        [121264, 121264],
        [121264, 121264]],

       [[121264, 121264],
        [121264, 121264],
        [121264, 121264],
        ...,
        [121264, 121264],
        [121264

In [18]:
incorrect = len(ds.time)
for i, jot in tqdm(enumerate(all_jets_over_time), total=len(all_jets_over_time)):
    jot = jot[jot[:, 0] != incorrect, :]
    props_over_time = extract_props_over_time(jot, props_as_ds_uncat)
    break
props_over_time

  0%|          | 0/60632 [00:00<?, ?it/s]


{'mean_lon': array([  3.73317,   6.07968,   9.0976 ,  13.06367, -30.22551, -35.17758,
         -6.26593,  -0.13182,   1.64339,  -4.03657,   5.07767]),
 'mean_lat': array([48.49671, 47.54169, 45.18422, 44.14048, 36.75577, 37.17177,
        37.85007, 36.7677 , 37.01632, 38.834  , 37.86554]),
 'mean_lev': array([236.03264, 244.56333, 248.02675, 244.69881, 250.65663, 248.19197,
        258.81276, 268.48601, 245.91983, 243.81845, 237.4432 ]),
 'is_polar': array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'Lon': array([  1.5,   3.5,  19. ,  24. , -75.5, -80. , -11.5, -20. ,  28.5,
         -6. ,  -1.5]),
 'Lat': array([51.5, 51.5, 37.5, 37. , 29.5, 29. , 37. , 37.5, 28.5, 34. , 34.5]),
 'Spe': array([59.06427, 56.02316, 56.16353, 56.12575, 75.78434, 83.44542,
        65.22881, 61.23854, 61.28957, 58.21903, 63.34647]),
 'lon_ext': array([ 22. ,  26.5,  28. ,  30.5, 109.5,  97.5,  59. ,  61. ,  56.5,
         39.5,  43. ]),
 'lat_ext': array([20.5, 23.5, 20.5, 23.5, 18.5, 20. , 20. , 20. ,